In [1]:
import warnings 
warnings.filterwarnings('ignore')

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("Airmean_chennai.csv")
df

,Date,City,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI
0,2015-01-01,Chennai,77.394417,149.388739,16.30,15.39,22.68,4.59,1.17,9.20,11.35000,0.170000,7.402406,1.197632,170.088957
1,2015-01-02,Chennai,77.394417,149.388739,16.49,13.42,23.09,7.83,1.23,8.61,9.16000,0.130000,7.402406,1.197632,170.088957
2,2015-01-03,Chennai,77.394417,149.388739,9.72,19.56,9.99,4.63,0.77,20.56,13.45000,0.030000,7.402406,1.197632,170.088957
3,2015-01-04,Chennai,77.394417,149.388739,9.60,16.20,11.71,5.23,1.00,20.56,10.33000,2.120333,7.402406,1.197632,170.088957
4,2015-01-05,Chennai,77.394417,149.388739,9.16,16.30,12.94,5.50,0.90,16.60,9.36000,2.120333,7.402406,1.197632,170.088957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,2020-06-27,Chennai,26.420000,83.998141,7.25,12.96,19.59,33.20,1.10,7.29,68.51000,0.100000,0.070000,1.197632,95.000000
2005,2020-06-28,Chennai,25.930000,83.998141,7.81,10.00,16.39,35.98,0.76,6.48,72.40625,0.090000,0.000000,1.197632,98.000000
2006,2020-06-29,Chennai,21.300000,83.998141,7.65,9.69,16.74,34.07,0.96,6.62,62.57000,0.090000,0.010000,1.197632,104.000000
2007,2020-06-30,Chennai,24.140000,83.998141,8.42,12.38,20.29,34.17,1.05,7.50,68.75000,0.170000,0.160000,1.197632,110.000000


In [8]:
df.isnull().sum()

Date       0
City       0
PM2.5      0
PM10       0
NO         0
NO2        0
NOx        0
NH3        0
CO         0
SO2        0
O3         0
Benzene    0
Toluene    0
Xylene     0
AQI        0
dtype: int64

In [9]:
df.isnull().sum()

In [10]:
timedata["Date"]=df["Date"]
timedata["AQI"]=df["AQI"]

In [11]:
timedata

,Date,AQI
0,2015-01-01,170.088957
1,2015-01-02,170.088957
2,2015-01-03,170.088957
3,2015-01-04,170.088957
4,2015-01-05,170.088957
...,...,...
2004,2020-06-27,95.000000
2005,2020-06-28,98.000000
2006,2020-06-29,104.000000
2007,2020-06-30,110.000000


In [12]:
df=timedata["AQI"]

In [15]:
df_train=df.head(1500)
df_test=df.tail(509)

In [16]:
from statsmodels.tsa.arima.model import ARIMA
ar=ARIMA(df_train, order=(2,0,1))
model = ar.fit()
y_pred = model.predict(len(df_train), len(df)-1)
print(y_pred)
from sklearn.metrics import r2_score
r2=r2_score(df_test,y_pred)
print("r2={}".format(r2))


1500    101.605235
1501    110.125728
1502    115.100989
1503    118.010617
1504    119.716672
           ...    
2004    126.026243
2005    126.027362
2006    126.028473
2007    126.029578
2008    126.030676
Name: predicted_mean, Length: 509, dtype: float64
r2=-0.7714393657078069


In [18]:
orders = [(0,0,0), (0,0,1), (2,0,1)]
orderslist = []
rscorelist = []

for i in orders:
    orderslist.append(i)
    
    from statsmodels.tsa.arima.model import ARIMA
    ar = ARIMA(df_train, order=i)  # FIX: use the actual tuple i
    model= ar.fit()
    
    y_pred = model.predict(len(df_train), len(df)-1)
    
    from sklearn.metrics import r2_score
    r2 = r2_score(df_test, y_pred)
    
    rscorelist.append(r2)
    print("Order={} ,r2={}".format(i, r2))


Order=(0, 0, 0) ,r2=-0.8177724549618632
Order=(0, 0, 1) ,r2=-0.8153375496178394
Order=(2, 0, 1) ,r2=-0.7714393657078069


In [19]:
rscorelist

[-0.8177724549618632, -0.8153375496178394, -0.7714393657078069]

In [20]:
result=pd.DataFrame()

In [22]:
result.insert(0,"Order",orderslist)
result.insert(1,"R_score",rscorelist)

In [23]:
result

,Order,R_score
0,"(0, 0, 0)",-0.817772
1,"(0, 0, 1)",-0.815338
2,"(2, 0, 1)",-0.771439


In [24]:
model.predict(len(df),len(df)+5)

2009    126.031767
2010    126.032851
2011    126.033928
2012    126.034999
2013    126.036062
2014    126.037119
Name: predicted_mean, dtype: float64